<a href="https://colab.research.google.com/github/pandser/task_of_classification/blob/main/spam_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Задача
Обучить модель на распознавание спама в русскоязычных коментариях.

---
###Реализация
Для перевода коментариев в используемом датасете применим библиотеку deep-translator.


In [ ]:
!pip install deep-translator

Импортируем необходимые библиотеки

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download("stopwords")

from deep_translator import GoogleTranslator
from joblib import dump
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Для обучения будет использоваться следующий [датасет](https://www.kaggle.com/datasets/ahsenwaheed/youtube-comments-spam-dataset?resource=download)

In [ ]:
df = pd.read_csv('Youtube-Spam-Dataset.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956 entries, 0 to 1955
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  1956 non-null   object
 1   AUTHOR      1956 non-null   object
 2   DATE        1711 non-null   object
 3   CONTENT     1956 non-null   object
 4   VIDEO_NAME  1956 non-null   object
 5   CLASS       1956 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 91.8+ KB


In [ ]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,VIDEO_NAME,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",PSY - GANGNAM STYLE(?????) M/V,1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,PSY - GANGNAM STYLE(?????) M/V,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,PSY - GANGNAM STYLE(?????) M/V,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,PSY - GANGNAM STYLE(?????) M/V,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,PSY - GANGNAM STYLE(?????) M/V,1


Проверим соотношение комментариев в исходных данных.

In [ ]:
df.CLASS.value_counts()

,count
CLASS,
1,1005
0,951


Оставим только интересующие нас данные, комментарии и класс (1 - спам, 0 - обычный комментарий)

In [ ]:
df = df[['CONTENT', 'CLASS']]
df.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


Добавим функцию для перевода комментариев, в случае неудачного перевода функция возвращает NaN.

In [ ]:
def eng_to_rus(text):
    try:
        return GoogleTranslator(source='en', target='ru').translate(text)
    except:
        return np.nan

Применяем функцию для перевода к датасету.

In [ ]:
df['CONTENT'] = df['CONTENT'].apply(eng_to_rus)

Проверим наличие пустых значений после перевода.

In [ ]:
df[df['CONTENT'].isnull()]

,CONTENT,CLASS
134,None,0
585,None,0
825,None,0
1386,None,0
1538,None,0
1821,None,0
1825,None,0


Удалим пустые данные из датасета.

In [ ]:
df = df.drop(df[df['CONTENT'].isnull()].index)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1949 entries, 0 to 1955
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CONTENT  1949 non-null   object
 1   CLASS    1949 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 45.7+ KB


Сохраним полученный датасет.

In [ ]:
df.to_csv('spam_comment_rus.csv')

Разделим полученные данные на тренировочный и тестовый наборы.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.CONTENT,
    df.CLASS,
    test_size=0.2,
)

Преобразуем данные в TF-IDF матрицы для тренировочного и тестового набора.

In [ ]:
tfidf_vectorizer=TfidfVectorizer(
    stop_words=stopwords.words("russian"),
    max_df=0.7,
)

In [ ]:
tfidf_train=tfidf_vectorizer.fit_transform(X_train)
tfidf_test=tfidf_vectorizer.transform(X_test)

Обучим модель используя пассивно-агрессивный алгоритм.

In [ ]:
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [ ]:
y_pred=pac.predict(tfidf_test)

Проверим точность прогнозирования на тестовых данных.

In [ ]:
score=accuracy_score(y_test,y_pred)
print(f'Точность: {round(score*100,2)}%')

Точность: 95.64%


Сохраним полученную модель.

In [ ]:
dump(pac, 'spam_filter.joblib')

['spam_filter.joblib']